In [1]:
import asyncio
import datetime
from time import sleep
import numpy as np
from pymeasure.experiment import Procedure, Worker, Results
from pymeasure.experiment import FloatParameter
import sys
sys.path.append("..\\")
from pylabinst.SR830 import SR830
from pylabsm.pylabsm_measure_procedures.pylabsm_proc_sr830 import run_sr830_procedure

In [6]:
class Sr830Procedure(Procedure):

    sr830_instance = None
    sample_frequency = FloatParameter("Sample Frequency", units="Hz.", default=4,
                                      minimum=2**-4, maximum=2**9)
    sample_time = FloatParameter("Sample Time", units="s.", default=10)

    DATA_COLUMNS = ["Time Stamp", "X Channel Voltage (V.)", "Y Channel Voltage (V.)"]

    def execute(self):
        if self.sr830_instance is not None:
            sample_period = 1/self.sample_frequency
            samples = int(np.ceil(self.sample_frequency * self.sample_time))
            print(samples)
            for i in range(samples):
                time_stamp = datetime.datetime.now()
                voltage = self.sr830_instance.snap().split(",")
                x_voltage = voltage[0]
                y_voltage = voltage[1]
                self.emit('results', {"Time Stamp": time_stamp, "X Channel Voltage (V.)": x_voltage,
                                      "Y Channel Voltage (V.)": y_voltage})
                sleep(sample_period)
        else:
            raise RuntimeError("No valid SR830 class instance has been passed to the procedure.")

In [8]:
sr830 = SR830()
await sr830.open()

filename = ".\\test.csv"
fs = 4
sample_time = 10

sr830_proc = Sr830Procedure()


results = Results(sr830_proc, filename)
worker = Worker(results)
sr830_proc.sr830_instance = sr830
sr830_proc.sample_frequency = fs
sr830_proc.sample_time = sample_time
worker.start()
worker.join(timeout=100)

40
